In [10]:
import numpy as np
import pandas as pd

import plotly.express as px

- Sigmoid/Logistic

    $$f(x) = \dfrac{1}{1 + e^{-x}}$$

    - Produz valores entre [0, 1]
    - Derivada: $f^{'}(x) = \text{sigmoid}(x) \times (1 - \text{sigmoid}(x))$

- Tangente hiperbólica
    - Parecida com a sigmoide, mas tem o range entre [-1, 1]

- Ambas possuem o problema de _vanishing gradient_
    - Muito do seu domínio

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dev_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [25]:
x = np.linspace(-10, 10, 200)
y = np.array([sigmoid(v) for v in x])

In [26]:
px.scatter(x=x, y=y, width=800)

In [27]:
y_ = np.array([dev_sigmoid(v) for v in x])
px.scatter(x=x, y=y_, width=800)

- Atualmente, ReLu (_Rectified Linear Unit_) ou alguma de suas variáveis são a escolha mais popular
    $$f(x) = max(0, x)$$

    - Simples e computacionalmente barata de se calcular
    - Em termos de otimização, também é mais simples

In [3]:
def relu(x):
    return max(0, x)


def dev_relu(x):
    return 1 if x >= 0 else 0

In [28]:
y = np.array([relu(v) for v in x])

px.scatter(x=x, y=y, width=800)

In [29]:
y_ = np.array([dev_relu(v) for v in x])

px.scatter(x=x, y=y_, width=800)

- Como todos os valores menores do zero tem gradiente nulo, durante a atualização da rede, alguns neurônios podem não ser atualizados
- Existe uma série de variações da ReLU que buscam mitigar seus problemas, enquanto mantendo as suas vantagens
- Lista de funções de ativação:

<img src="img/a6/activation.jpeg" height=1000px>
    <figcaption>Funções de ativação. Fonte: <a href="https://www.v7labs.com/blog/neural-networks-activation-functions">V7 Labs</a></figcaption>
</img>

<img src="img/a6/ann.jpeg" width=800px>
    <figcaption>Uma rede neural artificial. Fonte: <a href="https://www.v7labs.com/blog/neural-networks-activation-functions">V7 Labs</a></figcaption>
</img>

# Treinamento da ANN

- Backpropagation
    - Utiliza a regra da cadeia das derivadas para estimar como cada peso de neurônio contribui para o erro total da rede
    - 

- Época/epoca
    - "Passada" completa nos dados de treinamento
    - A ordem importa! É usual embaralhar os dados antes de cada rodada
- Dropout: regularização das redes
    - Desabilita alguns pesos durante o processo de otimização para encorajar a rede a se adaptar e, assim, evitar _overfitting_

In [4]:
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_validate, KFold

from sklearn.neural_network import MLPClassifier

In [5]:
X, y = load_digits(return_X_y=True, as_frame=True)

X.shape, np.unique(y)

((1797, 64), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [8]:
X.iloc[0]

pixel_0_0     0.0
pixel_0_1     0.0
pixel_0_2     5.0
pixel_0_3    13.0
pixel_0_4     9.0
             ... 
pixel_7_3    13.0
pixel_7_4    10.0
pixel_7_5     0.0
pixel_7_6     0.0
pixel_7_7     0.0
Name: 0, Length: 64, dtype: float64

In [6]:
ann_res = cross_validate(
    MLPClassifier(
        hidden_layer_sizes=(100),
        random_state=7,
    ),
    X, y,
    scoring=["accuracy", "precision_macro", "recall_macro", "f1_macro"],
    cv=KFold(n_splits=5, shuffle=True, random_state=123)
)

ann_res = pd.DataFrame(ann_res)

display(ann_res)
ann_res.mean(axis=0)

,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
0,1.511553,0.007795,0.980556,0.980922,0.981835,0.980798
1,1.685648,0.006663,0.963889,0.963203,0.963236,0.962831
2,1.644543,0.004923,0.980501,0.978622,0.979171,0.978541
3,1.223154,0.006930,0.983287,0.982977,0.983707,0.983040
4,1.377409,0.006110,0.980501,0.984763,0.980716,0.982143


fit_time                1.488461
score_time              0.006484
test_accuracy           0.977747
test_precision_macro    0.978097
test_recall_macro       0.977733
test_f1_macro           0.977471
dtype: float64

Visualização de fronteiras de decisão